In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import h5py
from sklearn.model_selection import train_test_split
import numpy as np
import keras.backend as K
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Convolution2D, Activation
from keras import backend as K
from keras.optimizers import SGD

def load():
    f = h5py.File("dataset.h5")
    x = f['x'].value
    y = f['y'].value
    f.close()
    x_train , x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=100)
    x_train = np.rollaxis(x_train, 1, 4)
    x_test = np.rollaxis(x_test, 1, 4)
    x_train = x_train  / 255.0
    x_test = x_test / 255.0
    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = load()

def multitask_loss(y_true, y_pred):
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    return K.mean(K.sum(- y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred), axis=1))

Using TensorFlow backend.


In [2]:
batch_size = 100
num_classes = 5
epochs = 50


# input image dimensions
img_rows, img_cols = 100, 100
channels = 3

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),padding='same',input_shape=(img_rows, img_cols, channels)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))


model.compile(loss=multitask_loss,
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 1600 samples, validate on 400 samples
Epoch 1/50
1600/1600 [==============================] - 6s 4ms/step - loss: 2.7115 - acc: 0.3275 - val_loss: 2.3993 - val_acc: 0.4275
Epoch 2/50
1600/1600 [==============================] - 2s 1ms/step - loss: 2.0483 - acc: 0.5375 - val_loss: 1.9218 - val_acc: 0.5050
Epoch 3/50
1600/1600 [==============================] - 2s 1ms/step - loss: 1.8621 - acc: 0.5888 - val_loss: 1.8554 - val_acc: 0.5925
Epoch 4/50
1600/1600 [==============================] - 2s 1ms/step - loss: 1.7560 - acc: 0.6112 - val_loss: 1.7801 - val_acc: 0.6300
Epoch 5/50
1600/1600 [==============================] - 2s 1ms/step - loss: 1.6126 - acc: 0.6406 - val_loss: 1.6801 - val_acc: 0.6475
Epoch 6/50
1600/1600 [==============================] - 2s 1ms/step - loss: 1.6353 - acc: 0.6338 - val_loss: 1.6856 - val_acc: 0.6350
Epoch 7/50
1600/1600 [==============================] - 2s 1ms/step - loss: 1.5145 - acc: 0.6494 - val_loss: 1.7114 - val_acc: 0.6300
Epoch 8/50
1600

In [34]:
def infer(input_data, model=model):
    inference = []
    
    classes = np.array(['desert', 'mountain', 'sea', 'sunset', 'trees'])
    
    y_pred = model.predict(input_data)
    
    # Performing masking
    y_pred = (y_pred > 0.5) * 1.0
    
    for i in range(y_pred.shape[0]):
        # select the indices
        indices = np.where(y_pred[i] == 1.0)[0]
        # Adding the results 
        inference.append(classes[indices].tolist())
        
    return inference

In [ ]:
model.save('multitask_model.h5')

In [35]:
infer(x_train, model=model)

[['sea', 'sunset'],
 ['sea'],
 ['mountain'],
 ['sea', 'sunset'],
 ['sunset'],
 ['trees'],
 ['sea'],
 ['trees'],
 ['mountain', 'trees'],
 ['sea'],
 ['sunset'],
 ['desert'],
 ['sea'],
 ['desert'],
 ['desert'],
 ['desert'],
 ['desert'],
 ['mountain'],
 ['trees'],
 ['mountain'],
 ['trees'],
 ['desert'],
 ['mountain'],
 ['sunset'],
 ['mountain'],
 ['sunset'],
 ['trees'],
 ['trees'],
 ['desert'],
 ['sea'],
 ['sea'],
 ['sunset', 'trees'],
 ['desert'],
 ['sea'],
 ['sunset'],
 ['sea'],
 ['mountain'],
 ['desert'],
 ['desert'],
 ['mountain', 'trees'],
 ['sea', 'sunset'],
 ['trees'],
 ['sea'],
 ['sunset'],
 ['trees'],
 ['sunset'],
 ['sunset'],
 ['mountain', 'trees'],
 ['desert'],
 ['mountain'],
 ['sea', 'sunset'],
 ['sea'],
 ['sea', 'sunset'],
 ['sea', 'sunset'],
 ['desert'],
 ['sea'],
 ['sea'],
 ['mountain'],
 ['trees'],
 ['mountain', 'trees'],
 ['sea'],
 ['sea'],
 ['sea'],
 ['sea', 'sunset'],
 ['trees'],
 ['sunset', 'trees'],
 ['sea', 'sunset'],
 ['sunset'],
 ['sea'],
 ['desert'],
 ['sea', 'suns